# Vitessce Widget Tutorial

# Visualization of a SpatialData object

## Import dependencies


In [1]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile
import shutil

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)

from vitessce.data_utils import (
    sdata_morton_sort_points,
    sdata_morton_query_rect,
)

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
import pandas as pd
import numpy as np
import json
import pyarrow.parquet as pq

In [3]:
from spatialdata import read_zarr, get_element_annotators, rasterize
from spatialdata.models import PointsModel

In [4]:
data_dir = "data"
zip_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr.zip")
spatialdata_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr")

In [5]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/xenium_rep1_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)
        
        # This Xenium dataset has an AnnData "raw" element.
        # Reference: https://github.com/giovp/spatialdata-sandbox/issues/55
        raw_dir = join(spatialdata_filepath, "tables", "table", "raw")
        if isdir(raw_dir):
            shutil.rmtree(raw_dir)

In [6]:
sdata = read_zarr(spatialdata_filepath)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

In [7]:
sdata["transcripts"].shape[0].compute()

42638083

In [8]:
sdata.tables["table"].X = sdata.tables["table"].X.toarray()

In [9]:
sdata.tables["dense_table"] = sdata.tables["table"]
sdata.write_element("dense_table")

In [10]:
# TODO: store the two separate images as a single image with two channels.
# Similar to https://github.com/EricMoerthVis/tissue-map-tools/pull/12

In [11]:
sdata.tables['table'].obs

,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area,region
0,1,28,1,0,29,58.387031,26.642188,cell_circles
1,2,94,0,0,94,197.016719,42.130781,cell_circles
2,3,9,0,0,9,16.256250,12.688906,cell_circles
3,4,11,0,0,11,42.311406,10.069844,cell_circles
4,5,48,0,0,48,107.652500,37.479688,cell_circles
...,...,...,...,...,...,...,...,...
167775,167776,229,1,0,230,220.452813,60.599688,cell_circles
167776,167777,79,0,0,79,37.389375,25.242344,cell_circles
167777,167778,397,0,0,397,287.058281,86.700000,cell_circles
167778,167779,117,0,0,117,235.354375,25.197188,cell_circles


In [12]:
"""
sdata["rasterized"] = rasterize(
    sdata["cell_boundaries"],
    ["x", "y"],
    min_coordinate=[0, 0],
    max_coordinate=[35416, 25778], # shape of image [x, y]
    target_coordinate_system="global",
    target_unit_to_pixels=1.0,
    return_regions_as_labels=True,
    value_key="cell_id",
)
"""

# Running into error:

"""
File .venv/lib/python3.10/site-packages/spatialdata/_core/operations/rasterize.py:723, in rasterize_shapes_points(data, axes, min_coordinate, max_coordinate, target_coordinate_system, target_unit_to_pixels, target_width, target_height, target_depth, element_name, sdata, value_key, table_name, return_regions_as_labels, agg_func, return_single_channel)
    721 max_uint16 = np.iinfo(np.uint16).max
    722 if max_label > max_uint16:
--> 723     raise ValueError(f"Maximum label index is {max_label}. Values higher than {max_uint16} are not supported.")
    724 agg = agg.astype(np.uint16)
    725 return Labels2DModel.parse(agg, transformations=transformations)

ValueError: Maximum label index is 167780. Values higher than 65535 are not supported.
"""

'\nFile .venv/lib/python3.10/site-packages/spatialdata/_core/operations/rasterize.py:723, in rasterize_shapes_points(data, axes, min_coordinate, max_coordinate, target_coordinate_system, target_unit_to_pixels, target_width, target_height, target_depth, element_name, sdata, value_key, table_name, return_regions_as_labels, agg_func, return_single_channel)\n    721 max_uint16 = np.iinfo(np.uint16).max\n    722 if max_label > max_uint16:\n--> 723     raise ValueError(f"Maximum label index is {max_label}. Values higher than {max_uint16} are not supported.")\n    724 agg = agg.astype(np.uint16)\n    725 return Labels2DModel.parse(agg, transformations=transformations)\n\nValueError: Maximum label index is 167780. Values higher than 65535 are not supported.\n'

In [13]:
sdata

SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      ├── 'dense_table': AnnData (167780, 313)
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

In [14]:
sdata.points['transcripts'].head()

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv
0,4.395842,328.666473,12.019493,SEC11C,565,0,281474976710656,18.662479
1,5.074415,236.964844,7.608510,NegControlCodeword_0502,540,0,281474976710657,18.634956
2,4.702023,322.797150,12.289083,SEC11C,562,0,281474976710658,18.662479
3,4.906601,581.428650,11.222615,DAPK3,271,0,281474976710659,20.821745
4,5.660699,720.851746,9.265523,TCIM,291,0,281474976710660,18.017488


In [15]:
sdata = sdata_morton_sort_points(sdata, "transcripts")

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/spatialdata/_core/_elements.py:115: UserWarning: Key `transcripts` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [16]:
# Add feature_index column to dataframe, and reorder columns so that feature_name (dict column) is the final column (all the way to the right)

In [17]:
# Convert feature_name column to feature_index column.
ddf = sdata.points['transcripts']
ddf.head()

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv,x_uint,y_uint,morton_code_2d
677718,16.058575,17.155981,6.504758,ERBB2,-1,0,281474977398719,40.000000,156,152,50128
542861,49.912006,13.121742,6.657761,TOMM7,-1,0,281474977261746,20.909292,451,104,96389
817519,50.067265,14.154890,6.614446,SERHL2,-1,0,281474977540499,40.000000,452,116,96816
600252,40.367912,24.953489,6.542048,SCD,-1,0,281474977320069,40.000000,367,246,114301
518878,56.408573,16.603086,6.755222,TPD52,-1,0,281474977237434,20.625040,507,146,120653


In [18]:
var_df = sdata.tables["table"].var
var_df.head()

,gene_ids,feature_types,genome
ABCC11,ENSG00000121270,Gene Expression,Unknown
ACTA2,ENSG00000107796,Gene Expression,Unknown
ACTG2,ENSG00000163017,Gene Expression,Unknown
ADAM9,ENSG00000168615,Gene Expression,Unknown
ADGRE5,ENSG00000123146,Gene Expression,Unknown


In [19]:
var_index = var_df.index.values.tolist()

In [20]:
# Add new feature_index column, then remove feature_name since Dictionary-encoded and not working with individual row group reading

def try_index(gene_name):
    try:
        return var_index.index(gene_name)
    except:
        return -1
ddf["feature_index"] = ddf["feature_name"].apply(try_index).astype('int32')

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/dask/dataframe/core.py:4448: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('feature_name', 'category'))

  warnings.warn(meta_warning(meta))


In [21]:
orig_columns = ddf.columns.tolist()
ordered_columns = sorted(orig_columns, key=lambda colname: orig_columns.index(colname) if colname != "feature_name" else len(orig_columns))
ordered_columns

['x',
 'y',
 'z',
 'cell_id',
 'overlaps_nucleus',
 'transcript_id',
 'qv',
 'x_uint',
 'y_uint',
 'morton_code_2d',
 'feature_index',
 'feature_name']

In [22]:
# Reorder the columns of the dataframe
ddf = ddf[ordered_columns]

In [23]:
ddf.head()

,x,y,z,cell_id,overlaps_nucleus,transcript_id,qv,x_uint,y_uint,morton_code_2d,feature_index,feature_name
677718,16.058575,17.155981,6.504758,-1,0,281474977398719,40.000000,156,152,50128,113,ERBB2
542861,49.912006,13.121742,6.657761,-1,0,281474977261746,20.909292,451,104,96389,294,TOMM7
817519,50.067265,14.154890,6.614446,-1,0,281474977540499,40.000000,452,116,96816,256,SERHL2
600252,40.367912,24.953489,6.542048,-1,0,281474977320069,40.000000,367,246,114301,250,SCD
518878,56.408573,16.603086,6.755222,-1,0,281474977237434,20.625040,507,146,120653,296,TPD52


In [24]:
# Save the updated points dataframe to disk

In [25]:
sdata["transcripts_with_morton_codes"] = ddf

In [26]:
sdata.write_element("transcripts_with_morton_codes")

In [27]:
# Update the row group size in each .parquet file part.
for i in range(8):
    table_read = pq.read_table(join(spatialdata_filepath, "points", "transcripts_with_morton_codes", "points.parquet", f"part.{i}.parquet"))

    # Write the table to a new Parquet file with the desired row group size.
    # Eventually, we would need a mechanism to pass the row_group_size parameter to where SpatialData calls DaskDataFrame.to_parquet (or change the default)
    # Reference: https://github.com/scverse/spatialdata/blob/aeef0cc3ebd7c10a7ed17b84415213bb259a5f4b/src/spatialdata/_io/io_points.py#L83C12-L83C22
    pq.write_table(
        table_read,
        join(spatialdata_filepath, "points", "transcripts_with_morton_codes", "points.parquet", f"part.{i}.parquet"),
        row_group_size=50_000,
        # You can also specify compression, version, etc.
        # compression='snappy',
        # version='2.6'
    )

In [28]:
# Note: this custom "bounding_box" attribute will not be written to disk:
# See https://github.com/scverse/spatialdata/blob/aeef0cc3ebd7c10a7ed17b84415213bb259a5f4b/src/spatialdata/_io/format.py#L116
sdata["transcripts"].attrs["bounding_box"]

{'x_min': -1.8734135627746582,
 'x_max': 7522.74609375,
 'y_min': 4.415738582611084,
 'y_max': 5473.509765625}

In [29]:
json.dumps(sdata["transcripts"].attrs["bounding_box"])

'{"x_min": -1.8734135627746582, "x_max": 7522.74609375, "y_min": 4.415738582611084, "y_max": 5473.509765625}'

In [30]:
# Check the number of row groups
parquet_file = pq.ParquetFile(join(spatialdata_filepath, "points", "transcripts_with_morton_codes", "points.parquet", "part.0.parquet"))

# Get the total number of row groups
num_groups = parquet_file.num_row_groups
num_groups

92

In [31]:
# DONE?

In [ ]:
sdata.points['transcripts_with_morton_codes'].head()

In [ ]:
ddf.head()

In [ ]:
ddf2 = ddf.drop(columns=["feature_name"])

In [ ]:
del ddf2.attrs["spatialdata_attrs"]["feature_key"]

In [ ]:
df = ddf2.compute()
df = df.reset_index()

In [ ]:
sdata.points["transcripts_with_morton_codes_without_feature_name"] = ddf2
sdata.write_element("transcripts_with_morton_codes_without_feature_name")

In [ ]:
ddf2.head()

In [ ]:
# Current row group size is ~1 million, or ~32 MB
# We want to try a row group size of ~50,000, or 1.6 MB

In [ ]:
for i in range(8):
    table_read = pq.read_table(join(spatialdata_filepath, "points", "transcripts_with_morton_codes_without_feature_name", "points.parquet", f"part.{i}.parquet"))

    # Write the table to a new Parquet file with the desired row group size.
    # Eventually, we would need a mechanism to pass the row_group_size parameter to where SpatialData calls DaskDataFrame.to_parquet (or change the default)
    # Reference: https://github.com/scverse/spatialdata/blob/aeef0cc3ebd7c10a7ed17b84415213bb259a5f4b/src/spatialdata/_io/io_points.py#L83C12-L83C22
    pq.write_table(
        table_read,
        join(spatialdata_filepath, "points", "transcripts_with_morton_codes_without_feature_name", "points.parquet", f"part.{i}.parquet"),
        row_group_size=50_000,
        # You can also specify compression, version, etc.
        # compression='snappy',
        # version='2.6'
    )

In [ ]:
# TODO: now upload the updated SpatialData object to S3

In [ ]:
sdata.points["transcripts"].head()

In [ ]:
sdata.points['transcripts'].attrs

In [ ]:
orig_rect = [[50, 50], [100, 150]] # x0, y0, x1, y1
rect_row_ranges = sdata_morton_query_rect(sdata, "transcripts", orig_rect)

In [ ]:
rect_row_ranges

In [ ]:
df = sdata.points["transcripts"].compute()

In [ ]:
# Convert list of (start, end) tuples to flat list of individual integer indices
row_indices = []
for (row_i, row_j) in rect_row_ranges:
    row_indices.extend(list(range(row_i, row_j)))

# Subset pandas df using matching row indices
df.iloc[row_indices, :]

In [ ]:
import dask.dataframe as dd

# Construct dask dataframe of points in range 100x200:
toy_df = pd.DataFrame(index=[], data=[], columns=["x", "y"])
toy_df["x"] = np.random.uniform(low=0.0, high=100.0, size=20)
toy_df["y"] = np.random.uniform(low=0.0, high=200.0, size=20)

toy_ddf = dd.from_pandas(toy_df, npartitions=2)

In [ ]:
# Compute morton codes
toy_ddf = norm_ddf_to_uint(toy_ddf)
toy_ddf["morton_code_2d"] = morton_interleave(toy_ddf)
sorted_ddf = toy_ddf.sort_values(by="morton_code_2d", ascending=True).compute()

## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.18",
    name='MERFISH SpatialData Demo',
)
# Add data to the configuration:
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/rasterized",
    table_path="tables/table",
    obs_feature_matrix_path="tables/table/X",
    obs_spots_path="shapes/cells",
    coordinate_system="global",
    coordination_values={
        # The following tells Vitessce to consider each observation as a "spot"
        "obsType": "cell",
    }
)
dataset = vc.add_dataset(name='MERFISH').add_object(wrapper)

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets = vc.add_view("obsSets", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'spotLayer': CL([{
        'obsType': 'cell',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSpots"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], [wrapper.obs_type_label])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller / obs_sets));

### Render the widget

In [ ]:
vw = vc.widget()
vw